# Fine-tuning a classifier using bounding box data from a 3LC Table

In this tutorial, we will fine-tune a classifier using bounding box data from a 3LC `Table`.

We will load the COCO128 table from an earlier notebook and use it to create a
`torch.utils.Dataset` of bounding box crops. These cropped images will be used to
fine-tune a classifier. In a later tutorial, we will use this trained model to
generate embeddings and predicted labels.

## Imports

In [ ]:
import tlc
from tlc_tools.augment_bbs.finetune_on_crops import train_model
from tlc_tools.common import infer_torch_device
from tlc_tools.split import split_table

## Project setup

In [ ]:
EPOCHS = 10
MODEL_CHECKPOINT = "../../../transient_data/bb_classifier.pth"
MODEL_NAME = "efficientnet_b0"
INCLUDE_BACKGROUND = False

## Set device

In [ ]:
DEVICE = infer_torch_device()
print(f"Using device: {DEVICE}")

## Load input Table

We will reuse the table created in the notebook [create-table-from-coco.ipynb](../../1-create-tables/create-table-from-coco.ipynb).

In [ ]:
input_table = tlc.Table.from_names(
    "initial",
    "COCO128",
    "3LC Tutorials",
)

## Split the Table

In [ ]:
# Create splits for training and validation
splits = split_table(input_table, {"train": 0.8, "val": 0.2})

train_table = splits["train"]
val_table = splits["val"]

print(f"Using table {train_table} for training")
print(f"Using table {val_table} for validation")

## Train model

In [ ]:
model, checkpoint_path = train_model(
    train_table_url=train_table.url,
    val_table_url=val_table.url,
    model_checkpoint=MODEL_CHECKPOINT,
    epochs=EPOCHS,
    include_background=INCLUDE_BACKGROUND,
    num_workers=0,  # Use more workers in a non-notebook environment
)